##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo entrenar un Deep Q Network con TF-Agents

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción


En este ejemplo se muestra cómo usar la biblioteca de TF-Agents para entrenar un agente [DQN (Deep Q Networks)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) en el entorno Cartpole.

![Cartpole environment](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

Le ayudará a familiarizarse con todos los componentes de un proceso de Aprendizaje por Refuerzo (RL) para el entrenamiento, la evaluación y la recopilación de datos.

Para ejecutar este código en directo, haga clic en el enlace "Ejecutar en Google Colab" que aparece más arriba.


## Preparación

Si todavía no ha instalado las siguientes dependencias, ejecute estos comandos:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

In [ ]:
# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION

## Hiperparámetros

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Entorno

En el Aprendizaje por Refuerzo (RL), un entorno representa la tarea o el problema a resolver. Los entornos estándar se pueden crear en TF-Agents con ayuda de los paquetes `tf_agents.environments`. TF-Agents tiene paquetes para cargar entornos desde fuentes como OpenAI Gym, Atari y DM Control.

Cargue el entorno CartPole desde el paquete de OpenAI Gym. 

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

Puede renderizar este entorno para ver cómo luce. Un poste que oscila libremente está unido a un carro. El objetivo es mover el carro a la derecha o a la izquierda para lograr que el poste siga apuntando hacia arriba.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

El método `environment.step` toma una `action` del entorno y devuelve una tupla `TimeStep` que contiene la siguiente observación del entorno y la recompensa correspondiente a la acción.

El método `time_step_spec()` devuelve la especificación para la tupla `TimeStep`. Su atributo `observation` muestra la forma de las observaciones, los tipos de datos y los intervalos de valores permitidos. El atributo `reward` muestra la misma información para la recompensa.


In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

El método `action_spec()` devuelve la forma, los tipos de datos y los valores permitidos de las acciones válidas.

In [ ]:
print('Action Spec:')
print(env.action_spec())

En el entorno Cartpole:

- `observation` es un arreglo de 4 flotantes:
    - la posición y la velocidad del carro
    - la posición angular y la velocidad del poste
- `reward` es un valor flotante escalar
- `action` es un entero escalar con solo dos valores posibles:
    - `0` — "mover a la izquierda"
    - `1` — "mover a la derecha"


In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Generalmente se crean instancias de dos entornos: uno para entrenamiento y otro para evaluación. 

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

El entorno Cartpole, al igual que la mayoría de los entornos, está escrito en Python puro. Este código se convierte a TensorFlow con ayuda del envoltorio `TFPyEnvironment`.

La API del entorno original usa arreglos Numpy. `TFPyEnvironment` los convierte en `Tensors` para que sea compatible con los agentes y las políticas de Tensorflow.


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

El algoritmo que se usa para resolver un problema de RL se representa por un `Agent`. TF-Agents ofrece implementaciones estándar de una variedad de `Agents`, lo que incluye lo siguiente:

- [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (usado en este tutorial)
- [REINFORCE](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
- [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
- [TD3](https://arxiv.org/pdf/1802.09477.pdf)
- [PPO](https://arxiv.org/abs/1707.06347)
- [SAC](https://arxiv.org/abs/1801.01290)

El agente DQN se puede usar en cualquier entorno que tenga un espacio de acción específico.

En el corazón de un Agente DQN hay una `QNetwork`, un modelo de red neuronal que puede aprender a predecir `QValues` (rendimientos esperados) para todas las acciones, a partir de una observación del entorno.

Usaremos `tf_agents.networks.` para crear una `QNetwork`. La red consistirá de una secuencia de capas de `tf.keras.layers.Dense`, donde la capa final tendrá 1 salida para cada acción posible.

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

Ahora, debe usar `tf_agents.agents.dqn.dqn_agent` para crear una instancia de `DqnAgent`. Además de `time_step_spec`, `action_spec` y QNetwork, el constructor del agente también necesita un optimizador (en este caso, `AdamOptimizer`), una función de pérdida y un contador de pasos de números enteros.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Políticas

Una política define el modo en que actúa un agente en un entorno. Normalmente, el objetivo del aprendizaje por refuerzo es entrenar el modelo subyacente hasta que la política produzca el resultado deseado.

En este tutorial:

- El resultado deseado es mantener el poste equilibrado en posición vertical sobre el carro.
- La política devuelve una acción (izquierda o derecha) para cada observación de `time_step`.

Los agentes contienen dos políticas:

- `agent.policy`: la política principal que se usa para la evaluación y la implementación.
- `agent.collect_policy`: una segunda política que se usa para la recopilación de datos.


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

Las políticas pueden crearse independientemente de los agentes. Por ejemplo, utilice `tf_agents.policies.random_tf_policy` para crear una política que seleccionará aleatoriamente una acción para cada `time_step`.

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

Para obtener una acción a partir de una política, llame al método `policy.action(time_step)`. `time_step` contiene la observación del entorno. Este método devuelve un `PolicyStep`, que es una tupla nombrada con tres componentes:

- `action`: la acción que debe tomarse (en este caso, `0` o `1`)
- `state`: se usa para políticas de estado (es decir, basadas en RNN)
- `info`: datos auxiliares, como las probabilidades logarítmicas de las acciones

In [ ]:
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [ ]:
time_step = example_environment.reset()

In [ ]:
random_policy.action(time_step)

## Métricas y evaluación

La métrica más común que se usa para evaluar una política es el rendimiento medio. El rendimiento es la suma de las recompensas obtenidas al ejecutar una política en un entorno durante un episodio. Se ejecutan varios episodios, lo que crea un rendimiento medio.

La siguiente función calcula el rendimiento medio de una política, para lo que se tiene en cuenta la política, el entorno y un número de episodios.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Ejecutar este cálculo en `random_policy` muestra un rendimiento de referencia en el entorno.

In [ ]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Búfer de repetición

Para hacer un seguimiento de los datos recogidos del entorno, se usará [Reverb](https://deepmind.com/research/open-source/Reverb), un sistema de repetición eficiente, extensible y fácil de usar de Deepmind. Este sistema almacena los datos de experiencia cuando recogemos trayectorias y se consume durante el entrenamiento.

Este búfer de repetición se construye a partir de especificaciones que describen los tensores que se van a almacenar, y que se pueden obtener del agente si se usa agent.collect_data_spec.


In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

Para la mayoría de los agentes, `collect_data_spec` es una tupla nombrada `Trajectory`, que contiene especificaciones de las observaciones, las acciones, las recompensas y otros elementos.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## Recopilación de datos

A continuación, ejecute la política aleatoria en el entorno durante unos pasos, registrando los datos en el búfer de repetición.

Aquí usamos 'PyDriver' para ejecutar el bucle de recopilación de experiencias. Si desea obtener más información sobre el controlador de TF Agents, consulte nuestro [tutorial sobre controladores](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

El búfer de repetición ahora es una colección de Trayectorias.

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

El agente necesita acceder al búfer de repetición. Para esto, se crea una canalización `tf.data.Dataset` iterable que ingresará datos al agente.

Cada fila del búfer de repetición almacena un único paso de observación. Pero dado que el agente DQN necesita tanto la observación actual como la siguiente para calcular la pérdida, la canalización del conjunto de datos extraerá dos filas adyacentes para cada elemento del lote (`num_steps=2`).

Este conjunto de datos también se optimiza mediante la ejecución de llamadas paralelas y la preextracción de datos.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:
iterator = iter(dataset)
print(iterator)

In [ ]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

## Entrenamiento del agente

Durante el bucle de entrenamiento tienen que ocurrir dos cosas:

- recopilación de datos del entorno
- uso de esos datos para entrenar la red neuronal del agente

Este ejemplo también evalúa periódicamente la política e imprime el puntaje actual.

Lo que sigue tardará unos ~5 minutos en ejecutarse.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualización


### Gráficos

Use `matplotlib.pyplot` para trazar la mejora de la política durante el entrenamiento.

Una iteración de `Cartpole-v0` consiste en 200 pasos de tiempo. El entorno ofrece una recompensa de `+1` por cada paso en el que el poste se mantiene erguido, por lo que la devolución máxima de un episodio corresponde a 200. Los gráficos muestran que la devolución aumenta hacia ese máximo cada vez que se evalúa durante el entrenamiento. (Es posible que sea un poco inestable y no siempre aumente monotónicamente).

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Videos

Los gráficos son agradables. Pero lo más emocionante es ver a un agente ejecutando una tarea en un entorno.

En primer lugar, cree una función para insertar videos en el bloc de notas.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Ahora iteramos a través de algunos episodios del juego Cartpole con el agente. El entorno Python subyacente (el que está "dentro" del envoltorio del entorno TensorFlow) dispone de un método `render()`, que muestra una imagen del estado del entorno. Estas imágenes pueden recopilarse en un video.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

Solo por diversión, compare el agente entrenado (arriba) con un agente que se mueve aleatoriamente. (Su rendimiento no es tan bueno).

In [ ]:
create_policy_eval_video(random_policy, "random-agent")